In [ ]:
from IPython.display import display, clear_output
clear_output(wait=True)

In [ ]:
# %matplotlib notebook
# 
# import os
# import time
# import matplotlib.pyplot as plt
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col, avg, explode, to_timestamp
# from pyspark.sql.types import StructType, StringType, IntegerType, DoubleType
# from pymongo import MongoClient
# import pandas as pd
# plt.ion()  # Turn on interactive mode
# 
# 
# # MongoDB connection
# hostip = "10.192.13.255"
# DB_NAME = "awas_db"
# 
# client = MongoClient(
#         host=f'{hostip}',
#         port=27017
#     )
# db = client["awas_db"]
# violation_col = db["violation"]
# 
# print(violation_col.count_documents({}))
# 
# for doc in violation_col.find().limit(5):
#     print(doc)
# 
# 
# os.environ["PYSPARK_SUBMIT_ARGS"] = (
#     "--packages "
#     "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0,"
#     "org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.0,"
#     "org.mongodb.spark:mongo-spark-connector_2.12:10.3.0 "
#     "pyspark-shell"
# )
# 
# # # Initialize Spark
# # spark = SparkSession.builder \
# #     .appName("AWAS-Speed-Enforcement") \
# #     .master("local[*]") \
# #     .config("spark.mongodb.read.connection.uri", f"mongodb://{hostip}:27017/{DB_NAME}") \
# #     .config("spark.mongodb.write.connection.uri", f"mongodb://{hostip}:27017/{DB_NAME}") \
# #     .getOrCreate()
# # spark.sparkContext.setLogLevel("WARN")
# # 
# # spark.read \
# #     .format("mongodb") \
# #     .option("collection", "violation") \
# #     .load()
# 
# # ====== Initialize plotting ======
# def init_plot():
#     fig = plt.figure(figsize=(9.5, 6))
#     fig.subplots_adjust(hspace=0.6)
#     ax1 = fig.add_subplot(221)
#     ax1.set_xlabel('Arrival Time')
#     ax1.set_ylabel('Number of Violations')
#     ax1.set_title('Violations vs Time')
#     ax2 = fig.add_subplot(222)
#     ax2.set_xlabel('Arrival Time')
#     ax2.set_ylabel('Average Speed')
#     ax2.set_title('Avg Speed vs Time')
#     fig.suptitle('AWAS Real-time Streaming Visualization')
#     fig.canvas.draw()
#     plt.pause(0.01)
# 
#     return fig, ax1, ax2
# 
# # ====== Annotations ======
# def annotate_max(x, y, ax):
#     ymax = max(y)
#     xpos = y.index(ymax)
#     ax.annotate(f'Max: {ymax}', xy=(x[xpos], ymax), xytext=(x[xpos], ymax+5),
#                 arrowprops=dict(facecolor='red', shrink=0.05))
# 
# def annotate_min(x, y, ax):
#     ymin = min(y)
#     xpos = y.index(ymin)
#     ax.annotate(f'Min: {ymin}', xy=(x[xpos], ymin), xytext=(x[xpos], ymin+5),
#                 arrowprops=dict(facecolor='orange', shrink=0.05))
# 
# def annotate_avg(x, y, ax):
#     avg_val = sum(y) / len(y)
#     ax.axhline(avg_val, color='blue', linestyle='--', linewidth=1)
#     ax.annotate(f'Avg: {avg_val:.2f}', xy=(x[-1], avg_val), xytext=(x[-1], avg_val + 5),
#                 arrowprops=dict(facecolor='blue', arrowstyle='->'), fontsize=9)
# 
# # def fetch_violation_data():
# #     pipeline = [
# #         {"$unwind": "$violations"},
# #         {"$project": {
# #             "_id": 0,
# #             "timestamp": "$violations.timestamp_start"
# #         }}
# #     ]
# #     cursor = violation_col.aggregate(pipeline)
# #     timestamps = [doc["timestamp"] for doc in cursor if "timestamp" in doc]
# #     return pd.DataFrame(timestamps, columns=["timestamp"])
# 
# def fetch_violation_data():
#     pipeline = [
#         {"$unwind": "$violations"},
#         {"$project": {
#             "timestamp": "$violations.timestamp_start",
#             "recorded_speed": "$violations.recorded_speed"
#         }}
#     ]
#     cursor = violation_col.aggregate(pipeline)
#     return pd.DataFrame(list(cursor))
# 
# def stream_plot_loop(fig, ax1, ax2, interval_sec=5):
#     while True:
#         print("Plotting tick...")  # Debug print
#         # ax1.set_title(f"Violations vs Time (tick {i})")
#         try:
#             df = fetch_violation_data()
# 
#             if df.empty:
#                 print("No data yet...")
#                 time.sleep(interval_sec)
#                 continue
# 
#             df["timestamp"] = pd.to_datetime(df["timestamp"], errors='coerce')
#             print(df["timestamp"].max())
# 
#             df.dropna(subset=["timestamp"], inplace=True)
# 
#             df["rounded_time"] = df["timestamp"].dt.floor("min")
# 
#             # === Aggregation ===
#             vdf = df.groupby("rounded_time").size().reset_index(name="num_violations")
#             sdf = df.groupby("rounded_time")["recorded_speed"].mean().reset_index(name="avg_speed")
# 
#             if vdf.empty or sdf.empty:
#                 time.sleep(interval_sec)
#                 continue
# 
#             # Clear plots
#             ax1.clear()
#             ax2.clear()
# 
#             # Re-label (since cla() removes them)
#             ax1.set_xlabel('Arrival Time')
#             ax1.set_ylabel('Number of Violations')
#             ax1.set_title('Violations vs Time')
#             ax1.set_title(f"Tick at {pd.Timestamp.now()}")
# 
#             ax2.set_xlabel('Arrival Time')
#             ax2.set_ylabel('Average Speed')
#             ax2.set_title('Avg Speed vs Time')
# 
#             # Plot violation count
#             x1, y1 = list(vdf["rounded_time"]), list(vdf["num_violations"])
#             ax1.plot(x1, y1, color='red')
#             annotate_max(x1, y1, ax1)
#             annotate_min(x1, y1, ax1)
# 
#             # Plot avg speed
#             x2, y2 = list(sdf["rounded_time"]), list(sdf["avg_speed"])
#             ax2.plot(x2, y2, color='blue')
#             annotate_max(x2, y2, ax2)
#             annotate_min(x2, y2, ax2)
#             annotate_avg(x2, y2, ax2)
# 
#             fig.canvas.draw_idle()
#             print(f"Plotted {len(x1)} points. Max time: {max(x1)}")
# 
#             plt.pause(0.01)
#             time.sleep(interval_sec)
# 
# 
# 
# 
#         except Exception as e:
#             print(f"[ERROR] {e}")
#             time.sleep(interval_sec)
# 
# 
# # ====== Run ======
# # if __name__ == "__main__":
# fig, ax1, ax2 = init_plot()
# plt.show()  # Show the plot without blocking
# stream_plot_loop(fig, ax1, ax2)
# 
# %matplotlib notebook
# 
# import time
# import matplotlib.pyplot as plt
# import pandas as pd
# from pymongo import MongoClient
# from datetime import datetime
# 
# # --- Setup ---
# plt.ion()
# hostip = "10.192.13.255"
# DB_NAME = "awas_db"
# client = MongoClient(host=hostip, port=27017)
# violation_col = client[DB_NAME]["violation"]
# 
# # --- Fetch from Mongo ---
# def fetch_violation_data():
#     pipeline = [
#         {"$unwind": "$violations"},
#         {"$project": {
#             "timestamp": "$violations.timestamp_start",
#             "recorded_speed": "$violations.recorded_speed"
#         }}
#     ]
#     return pd.DataFrame(list(violation_col.aggregate(pipeline)))
# 
# # --- Initialize plot ---
# def init_plot():
#     fig, ax1 = plt.subplots(figsize=(10, 6))
#     ax2 = ax1.twinx()
#     fig.suptitle("AWAS Real-time Streaming Visualization")
#     plt.show()
#     return fig, ax1, ax2
# 
# # --- Annotations ---
# def annotate_extremes(x, y, ax, label, color):
#     if not y: return
#     ymax = max(y)
#     ymin = min(y)
#     ax.annotate(f'Max {label}: {ymax:.1f}', xy=(x[y.index(ymax)], ymax),
#                 xytext=(x[y.index(ymax)], ymax + 2),
#                 arrowprops=dict(facecolor=color, shrink=0.05))
#     ax.annotate(f'Min {label}: {ymin:.1f}', xy=(x[y.index(ymin)], ymin),
#                 xytext=(x[y.index(ymin)], ymin - 4),
#                 arrowprops=dict(facecolor=color, shrink=0.05))
# 
# def annotate_avg(y, ax, color):
#     if not y: return
#     avg_val = sum(y) / len(y)
#     ax.axhline(avg_val, color=color, linestyle='--')
#     ax.annotate(f'Avg: {avg_val:.1f}', xy=(0.99, avg_val),
#                 xycoords=("axes fraction", "data"),
#                 ha="right", va="bottom",
#                 color=color, fontsize=9)
# 
# # --- Streaming loop ---
# def stream_plot_loop(fig, ax1, ax2, interval_sec=5):
#     while True:
#         try:
#             print("Fetching & plotting tick...")
#             df = fetch_violation_data()
#             df["timestamp"] = pd.to_datetime(df["timestamp"], errors='coerce')
#             df.dropna(subset=["timestamp"], inplace=True)
#             df["rounded_time"] = df["timestamp"].dt.floor("min")
# 
#             vdf = df.groupby("rounded_time").size().reset_index(name="num_violations")
#             sdf = df.groupby("rounded_time")["recorded_speed"].mean().reset_index(name="avg_speed")
# 
#             if vdf.empty or sdf.empty:
#                 time.sleep(interval_sec)
#                 continue
# 
#             # --- Clear and relabel ---
#             ax1.clear()
#             ax2.clear()
#             ax1.set_xlabel("Arrival Time")
#             ax1.set_ylabel("Violations", color="red")
#             ax2.set_ylabel("Avg Speed", color="blue")
# 
#             # --- Data ---
#             x_vals = list(vdf["rounded_time"])
#             y1_vals = list(vdf["num_violations"])
#             y2_vals = list(sdf["avg_speed"])
# 
#             # --- Plot ---
#             ax1.plot(x_vals, y1_vals, color="red", label="Violations")
#             ax2.plot(x_vals, y2_vals, color="blue", label="Avg Speed")
# 
#             annotate_extremes(x_vals, y1_vals, ax1, "Violations", "red")
#             annotate_extremes(x_vals, y2_vals, ax2, "Speed", "blue")
#             annotate_avg(y2_vals, ax2, "blue")
# 
#             ax1.set_title(f"Updated: {datetime.now().strftime('%H:%M:%S')}")
# 
#             fig.canvas.draw_idle()
#             plt.pause(0.01)
#             time.sleep(interval_sec)
# 
#         except Exception as e:
#             print(f"[ERROR]: {str(e)}")
#             time.sleep(interval_sec)
# 
# # --- Run ---
# fig, ax1, ax2 = init_plot()
# stream_plot_loop(fig, ax1, ax2)

%matplotlib notebook

import matplotlib.pyplot as plt
import pandas as pd
import time
from pymongo import MongoClient
from datetime import datetime

# Connect to MongoDB
hostip = "10.192.13.255"
client = MongoClient(host=hostip, port=27017)
violation_col = client["awas_db"]["violation"]

latest_time = "0000-00-00T00:00:00"  # Lowest lexicographically valid string
if not df.empty:
    latest_time = df["timestamp"].max()
    
# Fetch new violation records
def fetch_latest_data(latest_timestamp):
    pipeline = [
        {"$unwind": "$violations"},
        {"$project": {
            "timestamp": "$violations.timestamp_start",
            "recorded_speed": "$violations.recorded_speed"
        }},
        {"$match": {
            "timestamp": { "$gt": latest_timestamp }
        }}
    ]
    return pd.DataFrame(list(violation_col.aggregate(pipeline)))



# --- Setup plot ---
fig, ax1 = plt.subplots(figsize=(10, 5))
ax2 = ax1.twinx()

ax1.set_xlabel("Arrival Time")
ax1.set_ylabel("Violations", color="red")
ax2.set_ylabel("Avg Speed", color="blue")
fig.suptitle("Real-Time Growing Plot")

line1, = ax1.plot([], [], color='red', label="Violations")
line2, = ax2.plot([], [], color='blue', label="Avg Speed")

plt.show()

# --- Store accumulated history ---
history_df = pd.DataFrame(columns=["timestamp", "recorded_speed"])
latest_time = "2000-01-01T00:00:00"  # dummy old timestamp

while True:
    try:
        print("Polling MongoDB...")
        df = fetch_latest_data(latest_time)
        if df.empty:
            print("No new data.")
            time.sleep(3)
            continue

        # Clean and process
        df["timestamp"] = pd.to_datetime(df["timestamp"], errors='coerce')
        df.dropna(subset=["timestamp"], inplace=True)
        latest_time = df["timestamp"].max().isoformat()

        # Append new data to history
        history_df = pd.concat([history_df, df], ignore_index=True)
        history_df["rounded_time"] = history_df["timestamp"].dt.floor("min")

        # Aggregate
        vdf = history_df.groupby("rounded_time").size().reset_index(name="num_violations")
        sdf = history_df.groupby("rounded_time")["recorded_speed"].mean().reset_index(name="avg_speed")

        # Plot update
        line1.set_data(vdf["rounded_time"], vdf["num_violations"])
        line2.set_data(sdf["rounded_time"], sdf["avg_speed"])
        ax1.relim()
        ax2.relim()
        ax1.autoscale_view()
        ax2.autoscale_view()
        fig.canvas.draw_idle()
        plt.pause(0.01)
        time.sleep(3)

    except Exception as e:
        print("[ERROR]", e)
        time.sleep(3)




In [ ]:
# %matplotlib widget
# 
# import os
# import time
# import matplotlib.pyplot as plt
# import pandas as pd
# from pymongo import MongoClient
# from datetime import datetime
# 
# plt.ion()  # Interactive mode ON for live updates
# 
# # === MongoDB Setup ===
# hostip = "10.192.13.255"
# DB_NAME = "awas_db"
# 
# client = MongoClient(host=hostip, port=27017)
# violation_col = client[DB_NAME]["violation"]
# 
# # === Fetching helper ===
# def fetch_violation_data():
#     pipeline = [
#         {"$unwind": "$violations"},
#         {"$project": {
#             "timestamp": "$violations.timestamp_start",
#             "recorded_speed": "$violations.recorded_speed"
#         }}
#     ]
#     return pd.DataFrame(list(violation_col.aggregate(pipeline)))
# 
# # === Plot Setup ===
# def init_plot():
#     fig = plt.figure(figsize=(9.5, 6))
#     fig.subplots_adjust(hspace=0.6)
# 
#     ax1 = fig.add_subplot(221)
#     ax1.set_xlabel('Arrival Time')
#     ax1.set_ylabel('Number of Violations')
#     ax1.set_title('Violations vs Time')
# 
#     ax2 = fig.add_subplot(222)
#     ax2.set_xlabel('Arrival Time')
#     ax2.set_ylabel('Average Speed')
#     ax2.set_title('Avg Speed vs Time')
# 
#     fig.suptitle('AWAS Real-time Streaming Visualization')
#     plt.show()
#     return fig, ax1, ax2
# 
# # === Annotation Helpers ===
# def annotate_max(x, y, ax):
#     if not y: return
#     ymax = max(y)
#     xpos = y.index(ymax)
#     ax.annotate(f'Max: {ymax}', xy=(x[xpos], ymax), xytext=(x[xpos], ymax+5),
#                 arrowprops=dict(facecolor='red', shrink=0.05))
# 
# def annotate_min(x, y, ax):
#     if not y: return
#     ymin = min(y)
#     xpos = y.index(ymin)
#     ax.annotate(f'Min: {ymin}', xy=(x[xpos], ymin), xytext=(x[xpos], ymin+5),
#                 arrowprops=dict(facecolor='orange', shrink=0.05))
# 
# def annotate_avg(x, y, ax):
#     if not y: return
#     avg_val = sum(y) / len(y)
#     ax.axhline(avg_val, color='blue', linestyle='--', linewidth=1)
#     ax.annotate(f'Avg: {avg_val:.2f}', xy=(x[-1], avg_val), xytext=(x[-1], avg_val + 5),
#                 arrowprops=dict(facecolor='blue', arrowstyle='->'), fontsize=9)
# 
# # === Streaming Loop ===
# def stream_plot_loop(fig, ax1, ax2, interval_sec=5):
#     while True:
#         try:
#             df = fetch_violation_data()
#             if df.empty:
#                 print("Waiting for data...")
#                 time.sleep(interval_sec)
#                 continue
# 
#             df["timestamp"] = pd.to_datetime(df["timestamp"], errors='coerce')
#             df.dropna(subset=["timestamp"], inplace=True)
#             df["rounded_time"] = df["timestamp"].dt.floor("min")
# 
#             vdf = df.groupby("rounded_time").size().reset_index(name="num_violations")
#             sdf = df.groupby("rounded_time")["recorded_speed"].mean().reset_index(name="avg_speed")
# 
#             if vdf.empty or sdf.empty:
#                 time.sleep(interval_sec)
#                 continue
# 
#             # Clear and replot
#             ax1.clear()
#             ax2.clear()
# 
#             ax1.set_xlabel('Arrival Time')
#             ax1.set_ylabel('Number of Violations')
#             ax1.set_title(f"Violations vs Time @ {datetime.now().strftime('%H:%M:%S')}")
# 
#             ax2.set_xlabel('Arrival Time')
#             ax2.set_ylabel('Average Speed')
#             ax2.set_title('Avg Speed vs Time')
# 
#             # Plot data
#             x1, y1 = list(vdf["rounded_time"]), list(vdf["num_violations"])
#             ax1.plot(x1, y1, color='red')
#             annotate_max(x1, y1, ax1)
#             annotate_min(x1, y1, ax1)
# 
#             x2, y2 = list(sdf["rounded_time"]), list(sdf["avg_speed"])
#             ax2.plot(x2, y2, color='blue')
#             annotate_max(x2, y2, ax2)
#             annotate_min(x2, y2, ax2)
#             annotate_avg(x2, y2, ax2)
# 
#             fig.canvas.draw_idle()
#             plt.pause(0.01)
#             time.sleep(interval_sec)
# 
#         except Exception as e:
#             print(f"[ERROR]: {str(e)}")
#             time.sleep(interval_sec)
# 
# # === Run Once ===
# fig, ax1, ax2 = init_plot()
# stream_plot_loop(fig, ax1, ax2)
